In [0]:
import urllib

class S3Connector:
    """Utility to mount and unmount S3 bucket with given credentials."""
    
    def __init__(self, bucket_name, mount_name, credential_file):
        self.is_mounted = False
        self.bucket_name = bucket_name
        self.mount_name = mount_name
        creds = get_aws_credentials(credential_file)
        self.source_url = f"s3n://{creds[0]}:{creds[1]}@{self.bucket_name}"

    def mount(self):
        """Mount the S3 Bucket from DBFS"""
        mount_points = (mount.mountPoint for mount in dbutils.fs.mounts())
        if any(self.mount_name in mount for mount in mount_points):
            self.is_mounted = True
            print(f'{self.bucket_name} is already mounted.')
            return

        # When bucket is not already mounted and mounting is successful
        if not self.is_mounted and dbutils.fs.mount(self.source_url, self.mount_name):
            self.is_mounted = True
        else:
            raise Exception('Unable to mount the S3 Bucket')

    def unmount(self):
        """Unmount the S3 Bucket from DBFS"""
        if self.is_mounted and dbutils.fs.unmount(self.mount_name):
            self.is_mounted = False
        else:
            raise Exception('Unable to unmount.')
        

In [0]:
def get_aws_credentials(credentials, file_type='csv', header=True, sep=','):
    """Return the credentials to connect to S3 Bucket
    
    credentials: filename containing the AWS credentials
    file_type: specify file type, default 'csv'
    header: True indicates the file has first row as the header, else False
    sep: delimiter for the file, default comma ','
    """
    # Read the CSV file to spark dataframe
    # old method not used anymore
    # aws_keys_df = spark.read.format(file_type)\
    # .option("header", header)\
    # .option("sep", sep)\
    # .load(f"/FileStore/tables/{credentials}")

    # Define the path to the Delta table
    delta_table_path = "dbfs:/user/hive/warehouse/authentication_credentials"

    # Read the Delta table to a Spark DataFrame
    aws_keys_df = spark.read.format("delta").load(delta_table_path)
    
    # Get the AWS access key and secret key from the spark dataframe 
    ACCESS_KEY = aws_keys_df.select('Access key ID').collect()[0]['Access key ID'] 
    SECRET_KEY = aws_keys_df.select('Secret access key').collect()[0]['Secret access key'] # Encode the secrete key 
    ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")
    return ACCESS_KEY, ENCODED_SECRET_KEY, SECRET_KEY



In [0]:
def test_aws_connector():
    # creds = get_aws_credentials('authentication_credentials.csv')
    s3_conn = S3Connector('user-0a1d8948160f-bucket', '/mnt/pin_pipe', 'authentication_credentials.csv')
    assert not s3_conn.is_mounted
    s3_conn.mount()
    assert s3_conn.is_mounted
    s3_conn.unmount()
    assert not s3_conn.is_mounted

In [0]:
# test_aws_connector()

/mnt/pin_pipe has been unmounted.